# Interactive seq2seq
Applying our trained encoder-decoder model so that it can produce and respond to dialogue in context. We will need to load the model, seq2seq500embed, which was trained on the NLP cluster with 500 iterations and batch size of 1028.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings
import random
import json
from tensorflow.python.layers.core import Dense

In [2]:
import sys
sys.path.append("../src/models/")
sys.path.append('../src/models/agents/')
sys.path.append('../src/data/')
from agent import Agent
from parse import FBParser

In [3]:
train_iterations = 3
learning_rate = 0.1
max_input_length = 6 # TODO: change this
max_output_length = 20
unk_threshold = 20

### Initialization and loading
Need to initialize FBParser and Agent():

In [4]:
parser = FBParser(unk_threshold=unk_threshold,
                  input_directory="../data/raw/",
                  output_directory="../data/processed/")
print("Vocab size: {}".format(parser.vocab_size))

Vocab size: 502


In [52]:
agent = Agent(vocab=parser.vocab,
              max_iter=train_iterations,
              eta=learning_rate,
              max_input_length=max_input_length,
              max_output_length=max_output_length,
              hidden_dim=64)
agent.sess = tf.Session()

Load the model:

In [53]:
saver = tf.train.import_meta_graph('../models/seq2seq300embed.meta')
saver.restore(agent.sess, '../models/seq2seq300embed')

INFO:tensorflow:Restoring parameters from ../models/seq2seq300embed


In [54]:
test_data = []
with open("../data/processed/test.txt", "r") as test_file:
    for line in test_file:
        test_example = json.loads(line)
        test_data.append((
            test_example["input"],
            test_example["output"][0].split()))

In [55]:
X_test, y_test = zip(*test_data[:20])

In [56]:
X = X_test
X = np.asarray(list(X))
x_lengths = [len(seq) for seq in X] # len(seq) == 6
num_examples = len(X)
length = 6

# Resize X and x_lengths to match the size of inference_logits:
X.resize((agent.batch_size, length))
x_lengths = np.asarray(x_lengths)
x_lengths.resize(agent.batch_size)

graph = tf.get_default_graph()
answer_logits = agent.sess.run(graph.get_tensor_by_name("inference_logits:0"), {
    graph.get_tensor_by_name("encoder_inputs:0"): X, 
    graph.get_tensor_by_name("encoder_lengths:0"): x_lengths})[:num_examples]

In [57]:
print('\nPredictions:\n')
output = agent.output(answer_logits, padding=" ")
for i in range(len(output)):
    print(X_test[i])
    print(output[i])


Predictions:

[2, 2, 3, 2, 1, 0]
YOU: i can have the hat and the ball <eos> YOU: ok <eos> YOU: deal <eos> YOU: <selection>
[2, 0, 3, 1, 1, 7]
YOU: i can have the hat and the ball <eos> YOU: ok <eos> YOU: deal <eos> YOU: <selection>
[1, 1, 2, 3, 3, 1]
YOU: i can have the hat and the ball <eos> THEM: ok <eos> YOU: deal <eos> YOU: <selection>
[1, 10, 2, 0, 3, 0]
YOU: i can have the hat and the ball <eos> YOU: ok <eos> YOU: deal <eos> YOU: <selection>
[1, 1, 1, 5, 4, 1]
YOU: i can have the hat and the ball <eos> YOU: ok <eos> YOU: deal <eos> YOU: <selection>
[1, 9, 1, 1, 4, 0]
YOU: i can have the hat and the ball <eos> YOU: ok <eos> YOU: deal <eos> YOU: <selection>
[1, 1, 1, 3, 3, 2]
YOU: i can have the hat and the ball <eos> YOU: ok <eos> YOU: deal <eos> YOU: <selection>
[1, 1, 1, 0, 3, 3]
YOU: i can have the hat and the ball <eos> THEM: ok <eos> YOU: deal <eos> YOU: <selection>
[2, 2, 3, 2, 2, 0]
YOU: i can have the hat and the ball <eos> YOU: ok <eos> YOU: deal <eos> YOU: <selection>
[

In [40]:
def tokenize_string(string):
    tokens = string.strip().split()
    for i in range(len(tokens)):
        if tokens[i] in agent.vocab:
            tokens[i] = agent.vocab.index(tokens[i])
        else:
            tokens[i] = agent.vocab.index("$UNK")
    return tokens

In [45]:
X = [1, 0, 1, 5, 1, 5]

In [49]:
reply = tokenize_string("<eos> THEM: hello")

In [50]:
# Given a single example X:
def predict(X):
    length = len(X)
    X = [X]
    X = np.asarray(list(X))
    x_lengths = [len(seq) for seq in X] # len(seq) == 6
    print(length)
    print(x_lengths)
    

    # Resize X and x_lengths to match the size of inference_logits:
    X.resize((agent.batch_size, length))
    x_lengths = np.asarray(x_lengths)
    x_lengths.resize(agent.batch_size)

    graph = tf.get_default_graph()
    answer_logits = agent.sess.run(graph.get_tensor_by_name("inference_logits:0"), {
        graph.get_tensor_by_name("encoder_inputs:0"): X, 
        graph.get_tensor_by_name("encoder_lengths:0"): x_lengths})
    output = agent.output(answer_logits, padding=" ")
    return output[0]

In [51]:
new_string = X + reply
predict(new_string)

9
[9]


'YOU: i can have the book and the ball <eos> YOU: ok <eos> THEM: deal <eos> THEM: ok deal <eos>'

#### Found the issue: decoding_inference includes start_tokens, which come in between the encoder_inputs and the model output. Perhaps replace them with <eos\> or something like that.